In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_data = pd.read_json('../input/train.json')
test_data = pd.read_json('../input/test.json')
full_data = pd.concat([train,test])

#### Remove punctuation and numbers, stem words

In [ ]:
# Stemming is the process of reducing inflected (or sometimes derived) words to their word stem. 
from nltk.stem import PorterStemmer
import re

# Removes symbols, numbers and stem the words to reduce dimentional space
stemmer = PorterStemmer()

def clean(x):
    regex = re.compile('[^a-zA-Z ]')
    # For user clarity, broken it into three steps
    i = regex.sub(' ', x).lower()
    i = i.split(" ") 
    i= [stemmer.stem(l) for l in i]
    i= " ".join([l.strip() for l in i if (len(l)>2) ]) # Keeping words that have length greater than 2
    return i

# This takes some time to run. It would be helpful if someone can help me optimize clean() function.
full_data['description_new'] = full_data.description.apply(lambda x: clean(x))

#### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cvect_desc = CountVectorizer(stop_words='english',max_features=200)
full_sparse = cvect_desc.fit_transform(full_data.description_new)
# Renaming words to avoid collisions with other feature names in the model
col_desc = ['desc_'+ i for i in cvect_desc.get_feature_names()] 
count_vect_df = pd.DataFrame(full_sparse.todense(), columns=col_desc)
full_data = pd.concat([full_data.reset_index(),count_vect_df],axis=1)